In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark = SparkSession.builder.appName("SalesPrediction").getOrCreate()

In [14]:
# Load data into DataFrame
sales_data = spark.read.csv("C:/Users/hp/Downloads/archive/walmart-sales-dataset-of-45stores.csv", header=True, inferSchema=True)



In [16]:
print(sales_data)
sales_data.printSchema()

DataFrame[Store: int, Date: string, Weekly_Sales: double, Holiday_Flag: int, Temperature: double, Fuel_Price: double, CPI: double, Unemployment: double]
root
 |-- Store: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- Holiday_Flag: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: double (nullable = true)
 |-- Unemployment: double (nullable = true)



In [18]:
sales_data.show()

+-----+----------+------------+------------+-----------+----------+-----------+------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
|    1|05-02-2010|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|
|    1|12-02-2010|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|
|    1|19-02-2010|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|
|    1|26-02-2010|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|
|    1|05-03-2010|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|
|    1|12-03-2010|  1439541.59|           0|      57.79|     2.667|211.3806429|       8.106|
|    1|19-03-2010|  1472515.79|           0|      54.58|      2.72| 211.215635|       8.106|
|    1|26-03-2010|  1404429.92|           0|      51.45|     2.732|211

In [21]:
sales_data.isNull().show()

AttributeError: 'DataFrame' object has no attribute 'isNull'

In [24]:
cols_with_missing_values = [col for col in sales_data.columns if sales_data.select(col).where(sales_data[col].isNull()).count() > 0]
print(cols_with_missing_values)
print()


[]


In [30]:
cols_with_missing_values = []
for col in sales_data.columns:
    filtered_df = sales_data.select(col).where(sales_data[col].isNull())
    count = filtered_df.count()
    if count > 0:
        cols_with_missing_values.append(col)
print(f"Column with missing values : { cols_with_missing_values} ")


Column with missing values : [] 


In [33]:
sales_data.describe().show()

+-------+------------------+----------+------------------+-------------------+------------------+-------------------+-----------------+------------------+
|summary|             Store|      Date|      Weekly_Sales|       Holiday_Flag|       Temperature|         Fuel_Price|              CPI|      Unemployment|
+-------+------------------+----------+------------------+-------------------+------------------+-------------------+-----------------+------------------+
|  count|              6435|      6435|              6435|               6435|              6435|               6435|             6435|              6435|
|   mean|              23.0|      NULL|1046964.8775617732|0.06993006993006994| 60.66378243978229|  3.358606837606832|171.5783938487799| 7.999151048951067|
| stddev|12.988182381175454|      NULL| 564366.6220536977| 0.2550489443698279|18.444932875811585|0.45901970719285223|39.35671229566419|1.8758847818627944|
|    min|                 1|01-04-2011|         209986.25|            

In [34]:
#Correlation Analysis